In [1]:
import os
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import Text
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sentiment_analyzer = SentimentIntensityAnalyzer()
nltk.download('wordnet')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
from nltk.stem import PorterStemmer
stemmer= PorterStemmer()


import spacy
nlp = spacy.load('en_core_web_sm', disable = ['parser', 'ner'])

import gensim
from gensim import corpora
from gensim.models import Word2Vec


import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis


[nltk_data] Downloading package punkt to /home/felix/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/felix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/felix/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to /home/felix/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from textanalyzer import TextAnalyzer, metrics

/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-p

/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-p

In [3]:
def w2v_eng(input_text:str, model_name:str, count_min: int):
    lines = sent_tokenize(input_text)
    tokens = []
    for line in lines:
        tokens.append(gensim.utils.simple_preprocess(line))
    
    model = gensim.models.Word2Vec(
    window =  5,
    min_count = count_min,
    sg = 1,
    negative = 10
    )
    model.build_vocab(tokens)
    model.train(tokens, total_examples=model.corpus_count, epochs=model.epochs)
    model.save(f"{model_name}.model")
    

In [4]:
titles = ['Nicholas, Monsarrat - The Cruel Sea', 'The Cruel Sea - fragment', 'Maclean, Alistair - HMS Ulysses', 'Pikul, Valentin - Requiem to PQ-17',
          'Crane, Stephen - Red Badge of Courage', 'Bondarev, Yury - Hot Show']
all_texts_as_str = []
with open('Nicholas, Monsarrat - The Cruel Sea.txt', encoding='CP1252') as file:
    cruel_sea = file.read()
    all_texts_as_str.append(cruel_sea)


with open('The Cruel Sea - fragment.txt', encoding='CP1252') as file:
    cruel_sea_fragment = file.read()
    all_texts_as_str.append(cruel_sea_fragment)

with open('Maclean, Alistair - HMS Ulysses.txt', encoding='CP1252') as file:
    hms_ulysses = file.read()
    all_texts_as_str.append(hms_ulysses)
with open('Pikul, Valentin - Requiem to PQ-17.txt', encoding='utf-8') as file:
    requiem_pq17 = file.read()
    all_texts_as_str.append(requiem_pq17)

with open('Crane.txt', encoding='utf-8') as file:
    crane = file.read()
    all_texts_as_str.append(crane)
with open('Bondarev.txt', encoding='utf-8') as file:
    bondarev = file.read()
    all_texts_as_str.append(bondarev)

In [98]:
file_w = open ('text metrics.csv', mode = 'w', encoding='utf-8')
file_writer = csv.writer(file_w, delimiter = ";", lineterminator="\r")
file_writer.writerow(['title','number of words', 'number of sentences',
                    'number of unique words', 'number of paragraphs',
                    'average word length', 'average sentence length',
                    'average paragraph length in sentences',
                    'average paragraph length in words',
                    'sentiment negative', 'sentiment neutral',
                    'sentiment positive', 'sentiment compound'])


for index in range(len(all_texts_as_str)):
    novel = all_texts_as_str[index]
    text_metrics = metrics(novel)
    text_metrics.insert(0, titles[index])
    file_writer.writerow(text_metrics)

file_w.close()

In [99]:
compound_table = pd.read_csv('text metrics.csv', index_col='title', sep=";")
compound_table

,number of words,number of sentences,number of unique words,number of paragraphs,average word length,average sentence length,average paragraph length in sentences,average paragraph length in words,sentiment negative,sentiment neutral,sentiment positive,sentiment compound
title,,,,,,,,,,,,
"Nicholas, Monsarrat - The Cruel Sea",184158,9900,8507,7107,5.573562,18.601818,1.392993,25.912199,0.109,0.779,0.112,0.9998
The Cruel Sea - fragment,12115,565,2118,374,5.618489,21.442478,1.510695,32.393048,0.127,0.792,0.081,-1.0000
"Maclean, Alistair - HMS Ulysses",112931,8960,7653,5915,5.571827,12.603906,1.514793,19.092308,0.139,0.765,0.096,-1.0000
"Pikul, Valentin - Requiem to PQ-17",69558,4940,18646,5123,6.580738,14.080567,0.964279,13.577591,0.000,1.000,0.000,-0.6935
"Crane, Stephen - Red Badge of Courage",46677,3295,4836,1969,5.521992,14.166009,1.673438,23.705942,0.131,0.762,0.108,-1.0000
"Bondarev, Yury - Hot Show",83091,7337,19912,5413,6.891998,11.324928,1.355441,15.350268,0.000,1.000,0.000,0.0000


In [5]:
# here we need the model with parser so we change spacy core
nlp = spacy.load("en_core_web_sm")


/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-p

In [78]:
nlp.max_length = 1330000
doc = nlp(cruel_sea)
cruel_sea_entities = {}
for ent in doc.ents:
    cruel_sea_entities[ent.text] = ent.label_


In [16]:
doc = None

Named entities in *The Cruel Sea*

In [17]:
len(cruel_sea_entities)

1896

In [18]:
cruel_sea_entities

{'Nicholas Monsarrat': 'PERSON',
 '1951': 'DATE',
 'Atlantic': 'LOC',
 'Flag-Officer-in-Charge': 'ORG',
 'Liverpool': 'GPE',
 'Glasgow': 'GPE',
 'W.R.N.S.': 'GPE',
 'Clyde': 'GPE',
 '1943': 'DATE',
 'Hallam': 'PERSON',
 'Ardnacraish': 'GPE',
 'the Western Approaches Command': 'ORG',
 'one': 'CARDINAL',
 'two': 'CARDINAL',
 'about a hundred and fifty': 'CARDINAL',
 'a hundred and fifty': 'CARDINAL',
 'First': 'ORDINAL',
 'three': 'CARDINAL',
 'three thousand miles': 'QUANTITY',
 'European': 'NORP',
 'Africa': 'LOC',
 'American': 'NORP',
 'first': 'ORDINAL',
 'November 1939': 'DATE',
 'the hundred and fifty': 'CARDINAL',
 'at least a hundred and fifty': 'CARDINAL',
 'only heroines': 'CARDINAL',
 'ONE 1939': 'DATE',
 '1': 'CARDINAL',
 'George Eastwood Ericson': 'PERSON',
 'R.N.R.': 'ORG',
 'Fleming': 'ORG',
 'the River Clyde': 'FAC',
 'Ericson': 'PERSON',
 'about forty-two': 'CARDINAL',
 "twenty years'": 'DATE',
 "'Job No.": 'WORK_OF_ART',
 '2891': 'CARDINAL',
 "Movable Stores'": 'ORG',
 

In [19]:
entities = list(cruel_sea_entities.keys())
cruel_sea_locations = []
for entity in entities:
    if cruel_sea_entities[entity] == ('LOC' or 'GPE'):
        cruel_sea_locations.append(entity)

In [20]:
len(cruel_sea_locations)

59

In [21]:
cruel_sea_locations

['Atlantic',
 'Africa',
 'East-Coast',
 'the Far East Line',
 'the South Coast',
 'Watts',
 'the Black Sea',
 'Rounds',
 'nearest',
 'the Sea Lord',
 'Sorrel',
 'the growing moon',
 'the Bay of Biscay',
 'Liverpool Bay',
 'the Irish Sea',
 'Isle:',
 'West Country',
 'Hood',
 'Bismarck',
 'Baltic',
 'the Battle of the Atlantic',
 'trans-Atlantic',
 'mast',
 'Esquire',
 'Europe',
 'Mediterranean',
 'Inferno',
 'Straits',
 'Gibraltar harbour',
 'Gibraltar Harbour',
 'the dark sea',
 'Pacific',
 'the Bombed Ship',
 'Regent Street',
 'earth',
 'Central Station',
 'Whale Island',
 'East coast',
 'the east coast',
 'the Western Approaches',
 'West',
 'the South Atlantic',
 'the North Atlantic',
 'U-boats',
 'the Western Atlantic',
 'Holy Loch',
 'North Russia',
 'Bear Island',
 'North Cape',
 'Arctic',
 'the Arctic Circle',
 'Long Island Sound',
 'the East River',
 'Reckon',
 'Bunker Hill',
 'the Devon Coast',
 'Rhine',
 'Hostilities',
 'Transvaal']

Named entities for *HMS Ulysses*

In [22]:
doc = nlp(hms_ulysses)
hms_ulysses_entities = {}
for ent in doc.ents:
    hms_ulysses_entities[ent.text] = ent.label_
doc = None

In [23]:
len(hms_ulysses_entities)

1705

In [24]:
entities = list(hms_ulysses_entities.keys())
hms_ulysses_locations = []
for entity in entities:
    if hms_ulysses_entities[entity] == ('LOC' or 'GPE'):
        hms_ulysses_locations.append(entity)

In [25]:
len(hms_ulysses_locations)

55

In [26]:
hms_ulysses_locations

['Atlantic',
 'Arctic',
 'Scapa',
 'Bear Island',
 'the Sick Bay',
 'Burgess',
 'the Island of Coll',
 'Shetlands',
 'the Barents Sea',
 'South',
 'Mediterranean',
 'Sick Bay',
 'the North Cape',
 'Stockport',
 'Northern Norway',
 'the Roaring Forties',
 'the South Pacific',
 'Northern Lights',
 'south',
 'Norwegian Sea',
 'the Farmer Giles',
 'Horn',
 'mast',
 'East',
 'the Denmark Strait',
 'Caribbean',
 'the Arctic Circle',
 'the West Coast',
 'north-east',
 'the Tobacco Planter',
 'Planter',
 'the Bering Sea',
 'the Western Isles',
 'Pacific',
 'the Cape Hatteras',
 'U-boats',
 'the dark sea',
 'the Battery Room',
 'the Cape Hatterasl',
 'it?-he',
 'Ice',
 'the Kola Inlet',
 'Polyarnoe',
 'the Botanic Gardens',
 'West',
 'earth',
 'the Denmark Straits',
 'The Sick Bay',
 'North Atlantic',
 'the Jervis Bay',
 'the Northern Lights',
 'the Northern Lights-the Northern Lights',
 'Arctic night',
 'Bismarck',
 'The Northern Lights']

Named entities in *Requiem for PQ-17*

In [79]:
#lemmatization of Pikul's novel
nlp_rus = spacy.load("ru_core_news_lg")
doc = nlp_rus(requiem_pq17)
lemmas = [token.lemma_ for token in doc]
requiem_pq17_lemmatized = ' '.join(lemmas)


/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-packages/catalogue/__init__.py:135: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/home/felix/miniconda3/lib/python3.11/site-p

In [28]:
requiem_pq17_lemmatized

'в этой давней история мало беспристрастных свидетель — ещё меньше объективных судей . но слезы не высохли ... \n\n по вечерам еще скорбят старуха матери на зеленых берегах волги , темзы и миссисипи . \n\n но сначала мне хотеться сказать ... \n\n мой отец начать жизнь матрос на балтийский эсминец , а закончить ее комиссар морской пехота в руина сталинград . от него я перенять любовь к флот и юношеский тяга к стремительный корабль . сейчас мало кто знать , что с 1942 по 1945 год в наш флот существовать воинский звание — юнга . оно присваиваться подросток , которые освоить флотский специальность , дать воинский присяга и мочь наравне со взрослый нести самостоятельный вахта возле механизм . К число такой счастливец принадлежать и я . мне было 15 год , когда я стать рулевой на эскадренный миноносец . удивляться тут нечему : война — время большой доверие к юность . \n\n нам , юнга , очень хотеться попасть в самую заваруха морской война , и мне здорово повезти — я служить на северный флот . 

In [29]:
doc_lemmatized = nlp_rus(requiem_pq17_lemmatized)
requiem_pq17_entities = {}
for ent in doc_lemmatized.ents:
    requiem_pq17_entities[ent.text] = ent.label_
doc = None

In [30]:
len(requiem_pq17_entities)

624

In [31]:
entities = list(requiem_pq17_entities.keys())
requiem_pq17_locations = []
for entity in entities:
    if requiem_pq17_entities[entity] == ('LOC' or 'GPE'):
        requiem_pq17_locations.append(entity)

In [32]:
len(requiem_pq17_locations)

220

In [33]:
requiem_pq17_locations

['волги',
 'темзы',
 'миссисипи',
 'сталинград',
 'курск',
 'арктика',
 'майен',
 'медвежьим',
 'исландия',
 'берлин',
 'гдыня',
 'англия',
 'европа',
 'стокгольм',
 'лондон',
 'германия',
 'гибралтар',
 'африка',
 'южный америка',
 'норвегия',
 'далёкий родина',
 'корс - фьорд',
 'берген',
 'англию',
 'ссср',
 'москва',
 'россия',
 'америка',
 'датский пролив',
 'брест',
 'британский остров',
 'атлантический океан',
 'сша',
 'атлантика',
 'северный полюс',
 'антарктика',
 'варшава',
 'пирл - харбор',
 'флорида',
 'майами',
 'гавана',
 'ньюфаундленд',
 'мексика',
 'гвиана',
 'португалия',
 'бермудский остров',
 'ла - манш',
 'франция',
 'па - де - кале',
 'северный море',
 'сингапур',
 'анкара',
 'турция',
 'старый руссы',
 'нью - йорк',
 'советский союз',
 'аас - фьорд',
 'тронхейма',
 'ла - манша',
 'бреста',
 'сомма',
 'кале',
 'дувр',
 'нарвика',
 'архангельск',
 'иран',
 'владивосток',
 'сибирь',
 'япония',
 'мурманск',
 'клин',
 'скапа - флоу',
 'готенхафена',
 'керченский полуос

### Word2Vec for *The Cruel Sea*

In [57]:
w2v_eng(cruel_sea, 'w2v_cruel_sea', 4)
model = Word2Vec.load("w2v_cruel_sea.model")

In [35]:
model.wv.most_similar("cold")

[('lined', 0.9476543664932251),
 ('stiff', 0.9472423195838928),
 ('growing', 0.9462230205535889),
 ('flicker', 0.9459898471832275),
 ('green', 0.9458039402961731),
 ('violence', 0.9450666904449463),
 ('thick', 0.9439725279808044),
 ('lit', 0.9428666830062866),
 ('skin', 0.9420546889305115),
 ('hull', 0.9413630366325378)]

In [37]:
#model.wv.most_similar("arctic")

KeyError: "Key 'arctic' not present in vocabulary"

In [58]:
#model.wv.most_similar("polar")

KeyError: "Key 'polar' not present in vocabulary"

In [39]:
model.wv.most_similar("ice")

[('slapping', 0.995128333568573),
 ('carpet', 0.9947869777679443),
 ('wood', 0.9943699240684509),
 ('racing', 0.994086742401123),
 ('lamps', 0.9939538240432739),
 ('yard', 0.9937460422515869),
 ('portion', 0.9936249852180481),
 ('streaked', 0.9935423135757446),
 ('sunlight', 0.9935110807418823),
 ('slopping', 0.9934109449386597)]

In [8]:
model.wv.most_similar("war")

[('thing', 0.9209874868392944),
 ('navy', 0.9187304377555847),
 ('job', 0.9167075753211975),
 ('worst', 0.9157894849777222),
 ('world', 0.9073374271392822),
 ('weather', 0.9055261015892029),
 ('future', 0.9020546078681946),
 ('part', 0.9018517732620239),
 ('year', 0.8908471465110779),
 ('story', 0.890216052532196)]

In [9]:
model.wv.most_similar("russia")

[('qualified', 0.9960533976554871),
 ('language', 0.9958796501159668),
 ('scrounge', 0.9957273602485657),
 ('relay', 0.9954387545585632),
 ('cows', 0.9952930808067322),
 ('prefer', 0.9951840043067932),
 ('idle', 0.9950882792472839),
 ('ferrabee', 0.9950597882270813),
 ('cargo', 0.9950242638587952),
 ('hospitality', 0.9949241876602173)]

In [10]:
model.wv.most_similar("north")

[('anchor', 0.9619129300117493),
 ('mile', 0.9599312543869019),
 ('south', 0.9582234621047974),
 ('steam', 0.9580193161964417),
 ('ahead', 0.9579405188560486),
 ('eastwards', 0.9567760825157166),
 ('steamed', 0.9565079808235168),
 ('guns', 0.9547791481018066),
 ('mooring', 0.9547483921051025),
 ('steaming', 0.9536454081535339)]

In [11]:
model.wv.most_similar("arctic")

[('specialist', 0.9960625171661377),
 ('possibilities', 0.9954416155815125),
 ('assorted', 0.9954315423965454),
 ('dangers', 0.9952749609947205),
 ('boxes', 0.9952636957168579),
 ('depressed', 0.995107114315033),
 ('apparently', 0.9950643181800842),
 ('shallow', 0.9949715733528137),
 ('featured', 0.994922935962677),
 ('chancing', 0.994915783405304)]

In [39]:
model.wv.most_similar("sea")

[('speed', 0.9079800844192505),
 ('harbour', 0.8658565282821655),
 ('line', 0.8620948791503906),
 ('area', 0.8586443066596985),
 ('level', 0.8520680665969849),
 ('angle', 0.8502627015113831),
 ('distance', 0.8497998118400574),
 ('anchor', 0.8428811430931091),
 ('atlantic', 0.8420650362968445),
 ('liverpool', 0.8417853713035583)]

In [52]:
model.wv.most_similar("gale")

[('fleet', 0.9853932857513428),
 ('rendezvous', 0.9853870272636414),
 ('quarters', 0.9852606654167175),
 ('perfect', 0.9847234487533569),
 ('loss', 0.9845102429389954),
 ('exercise', 0.9836457371711731),
 ('death', 0.9829027056694031),
 ('activity', 0.9824978709220886),
 ('flame', 0.9822404980659485),
 ('stars', 0.9821484088897705)]

### Word2Vec for the Fragment of The Cruel Sea

In [53]:
w2v_eng(cruel_sea, 'w2v_cruel_sea_fragment', 2)
model = Word2Vec.load("w2v_cruel_sea_fragment.model")

In [7]:
model.wv.most_similar("cold")

[('empty', 0.9406285285949707),
 ('thin', 0.9406140446662903),
 ('wild', 0.940300703048706),
 ('strong', 0.9372853636741638),
 ('thick', 0.9333316683769226),
 ('itself', 0.9329465627670288),
 ('shadow', 0.932741641998291),
 ('moon', 0.930385410785675),
 ('lifting', 0.9294941425323486),
 ('showing', 0.9290540814399719)]

In [28]:
model.wv.most_similar("ice")

[('flesh', 0.9969170093536377),
 ('cottages', 0.9967816472053528),
 ('columns', 0.9966172575950623),
 ('battered', 0.996576189994812),
 ('frosty', 0.9961395859718323),
 ('sparkling', 0.9956384897232056),
 ('scraps', 0.995278537273407),
 ('pavement', 0.9950689077377319),
 ('patches', 0.9950388669967651),
 ('film', 0.9950205683708191)]

In [30]:
model.wv.most_similar("arctic")

[('specialist', 0.9963591694831848),
 ('shallow', 0.9963417649269104),
 ('patrolling', 0.9959845542907715),
 ('awareness', 0.9958295822143555),
 ('pondered', 0.9956392645835876),
 ('widow', 0.9956023097038269),
 ('lascars', 0.995596706867218),
 ('chocks', 0.9954278469085693),
 ('dangers', 0.995416522026062),
 ('despite', 0.9953885078430176)]

In [32]:
model.wv.most_similar("north")

[('eastwards', 0.9792650938034058),
 ('mile', 0.9769070744514465),
 ('steam', 0.9738271236419678),
 ('east', 0.9731936454772949),
 ('gladstone', 0.9729871153831482),
 ('main', 0.9712743759155273),
 ('irish', 0.9697507619857788),
 ('tide', 0.9688540101051331),
 ('westwards', 0.9686744809150696),
 ('flat', 0.9681057333946228)]

In [33]:
model.wv.most_similar("russia")

[('glorified', 0.995995283126831),
 ('costing', 0.9957574009895325),
 ('growth', 0.9955594539642334),
 ('weighing', 0.9953863024711609),
 ('qualified', 0.9953637719154358),
 ('overwork', 0.9953547120094299),
 ('possibility', 0.9953413605690002),
 ('tame', 0.9953240752220154),
 ('scrounge', 0.9952088594436646),
 ('hamilton', 0.9951989650726318)]

In [34]:
model.wv.most_similar("war")

[('thing', 0.9283698201179504),
 ('job', 0.9252935647964478),
 ('part', 0.920386552810669),
 ('worst', 0.9169324636459351),
 ('world', 0.9093157649040222),
 ('navy', 0.905001163482666),
 ('whole', 0.9009430408477783),
 ('luck', 0.9005148410797119),
 ('life', 0.8996052742004395),
 ('escort', 0.8992611765861511)]

In [35]:
model.wv.most_similar("polar")

KeyError: "Key 'polar' not present in vocabulary"

In [36]:
model.wv.most_similar("sea")

[('speed', 0.9174847602844238),
 ('land', 0.8793772459030151),
 ('line', 0.8792979121208191),
 ('harbour', 0.8776353597640991),
 ('irish', 0.8756716847419739),
 ('lay', 0.8749022483825684),
 ('steaming', 0.8722544312477112),
 ('atlantic', 0.871442437171936),
 ('convoy', 0.8710721731185913),
 ('fall', 0.8676889538764954)]

In [54]:
model.wv.most_similar("gale")

[('uncomfortable', 0.9896061420440674),
 ('mark', 0.9887917637825012),
 ('exercises', 0.9884124398231506),
 ('crucial', 0.9868357181549072),
 ('hated', 0.9867912530899048),
 ('struggle', 0.9863002300262451),
 ('swell', 0.9862083792686462),
 ('spring', 0.9861496090888977),
 ('loss', 0.9859108328819275),
 ('sheltered', 0.9858208298683167)]

### Word2Vec for HMS Ulysses

In [59]:
w2v_eng(hms_ulysses, 'w2v_hms_ulysses', 4)
model = Word2Vec.load("w2v_hms_ulysses.model")

In [42]:
model.wv.most_similar("cold")

[('deadly', 0.9832672476768494),
 ('force', 0.9827771782875061),
 ('hundreds', 0.9826004505157471),
 ('single', 0.9824075102806091),
 ('near', 0.9822172522544861),
 ('reports', 0.9814309477806091),
 ('boats', 0.9812710881233215),
 ('german', 0.9812684655189514),
 ('fast', 0.9812273383140564),
 ('ton', 0.9812193512916565)]

In [43]:
model.wv.most_similar("arctic")

[('surface', 0.9872124791145325),
 ('turrets', 0.9858956336975098),
 ('guns', 0.9857187867164612),
 ('condor', 0.9855534434318542),
 ('roaring', 0.9845214486122131),
 ('turret', 0.9841028451919556),
 ('superstructure', 0.9835458993911743),
 ('wave', 0.9834383726119995),
 ('gun', 0.9831542372703552),
 ('line', 0.9827406406402588)]

In [44]:
model.wv.most_similar("north")

[('east', 0.986946702003479),
 ('sirrus', 0.9803799390792847),
 ('bottom', 0.9778391718864441),
 ('run', 0.9778034687042236),
 ('stirling', 0.9773188829421997),
 ('west', 0.9767094254493713),
 ('squadron', 0.9755465984344482),
 ('home', 0.9754262566566467),
 ('end', 0.9753036499023438),
 ('wheelhouse', 0.9748846292495728)]

In [45]:
model.wv.most_similar("polar")

[('leaping', 0.9985395073890686),
 ('stem', 0.9985004663467407),
 ('darkened', 0.9984787106513977),
 ('viscous', 0.9984718561172485),
 ('opening', 0.9984429478645325),
 ('patches', 0.998441755771637),
 ('cloudless', 0.9984211921691895),
 ('swirling', 0.9983645677566528),
 ('streaming', 0.9983230829238892),
 ('appalling', 0.9983190894126892)]

In [46]:
model.wv.most_similar("ice")

[('black', 0.9926164746284485),
 ('poop', 0.9912740588188171),
 ('engines', 0.9907920360565186),
 ('shattered', 0.9905051589012146),
 ('darkness', 0.9903828501701355),
 ('crashed', 0.9898335337638855),
 ('rolling', 0.9893270134925842),
 ('steel', 0.9887705445289612),
 ('falling', 0.9883671998977661),
 ('sky', 0.9883000254631042)]

In [47]:
model.wv.most_similar("war")

[('little', 0.9941266179084778),
 ('clearly', 0.9933000802993774),
 ('human', 0.9928377270698547),
 ('alone', 0.9924004673957825),
 ('live', 0.9923837184906006),
 ('mistaking', 0.9923059940338135),
 ('sense', 0.9921559691429138),
 ('blind', 0.9919400215148926),
 ('convoys', 0.9918041229248047),
 ('fact', 0.9913301467895508)]

In [45]:
model.wv.most_similar("sea")

[('seas', 0.9851323366165161),
 ('bows', 0.9848774075508118),
 ('surface', 0.9761164784431458),
 ('mess', 0.9753340482711792),
 ('rolling', 0.9737719893455505),
 ('decks', 0.9730908870697021),
 ('ice', 0.97207111120224),
 ('water', 0.9714810848236084),
 ('smoke', 0.9691236019134521),
 ('great', 0.9679029583930969)]

In [56]:
model.wv.most_similar("gale")

[('howling', 0.9978591203689575),
 ('defence', 0.997802197933197),
 ('asleep', 0.9976341128349304),
 ('total', 0.9976083636283875),
 ('cunning', 0.9975619316101074),
 ('clothes', 0.9974109530448914),
 ('unmistakable', 0.9973790049552917),
 ('code', 0.9973275065422058),
 ('perfect', 0.9972527623176575),
 ('drowned', 0.9972497820854187)]

In [60]:
model.wv.most_similar("russia")

[('boys', 0.997450053691864),
 ('courage', 0.9969927668571472),
 ('cumberland', 0.9968686103820801),
 ('isolated', 0.9968476295471191),
 ('charges', 0.9966446161270142),
 ('capital', 0.9965450763702393),
 ('gunfire', 0.9965199828147888),
 ('packs', 0.9964750409126282),
 ('alta', 0.9962896108627319),
 ('fine', 0.9960851669311523)]

### Word2Vec for Requiem PQ-17

In [46]:
from pymystem3 import Mystem

# Initialize Mystem
m = Mystem()

lemmas = m.lemmatize(requiem_pq17)
lemmatized_pikul = ''.join(lemmas)

In [61]:
lines = sent_tokenize(lemmatized_pikul)
tokens = []
for line in lines:
    tokens.append(gensim.utils.simple_preprocess(line))

    


In [62]:
model = gensim.models.Word2Vec(
window =  5,
min_count = 5,
sg = 1,
negative = 10
)
model.build_vocab(tokens)
model.train(tokens, total_examples=model.corpus_count, epochs=model.epochs)

(201149, 298135)

In [16]:
model.wv.most_similar("арктика")

[('великолепный', 0.9990298748016357),
 ('ощущение', 0.9989566206932068),
 ('ад', 0.9989463090896606),
 ('щит', 0.9989349842071533),
 ('составлять', 0.9989275336265564),
 ('роковой', 0.9989223480224609),
 ('выкрикивать', 0.9989177584648132),
 ('трястись', 0.9989174604415894),
 ('отправляться', 0.9989119172096252),
 ('горячий', 0.998905599117279)]

In [17]:
model.wv.most_similar("лед")

[('струя', 0.9977277517318726),
 ('кромка', 0.9976992011070251),
 ('среди', 0.9976590871810913),
 ('высоко', 0.9975379705429077),
 ('каталина', 0.9975231885910034),
 ('красный', 0.9974530339241028),
 ('вырастать', 0.9974319934844971),
 ('громадный', 0.9974048733711243),
 ('дым', 0.9974000453948975),
 ('танкер', 0.9973760843276978)]

In [18]:
model.wv.most_similar("мороз")

[('обгорелый', 0.9990355372428894),
 ('дымить', 0.9989418387413025),
 ('грудь', 0.9989272356033325),
 ('камень', 0.9989098906517029),
 ('поле', 0.9989098310470581),
 ('кулак', 0.9989092946052551),
 ('обходить', 0.998906135559082),
 ('осколок', 0.9988993406295776),
 ('цистерна', 0.9988973736763),
 ('переводить', 0.9988903999328613)]

In [19]:
model.wv.most_similar("холодный")

[('кабина', 0.9991117119789124),
 ('назад', 0.9990473389625549),
 ('завывать', 0.9990100264549255),
 ('поросенок', 0.9989739060401917),
 ('глава', 0.998958945274353),
 ('прозвучать', 0.9989070892333984),
 ('кресло', 0.9989027976989746),
 ('треск', 0.9988992810249329),
 ('открывать', 0.9988991022109985),
 ('стальной', 0.9988906979560852)]

In [20]:
model.wv.most_similar("полярный")

[('вырастать', 0.9958996176719666),
 ('быстро', 0.9955419301986694),
 ('киль', 0.9954929947853088),
 ('воздух', 0.9953327775001526),
 ('поверхность', 0.9951542019844055),
 ('почти', 0.9951069355010986),
 ('блуждать', 0.99502032995224),
 ('вести', 0.9949438571929932),
 ('черный', 0.9949245452880859),
 ('высоко', 0.9948354363441467)]

In [23]:
model.wv.most_similar("северный")

[('германский', 0.9864025115966797),
 ('гитлеровский', 0.9842340350151062),
 ('английский', 0.9837476015090942),
 ('командование', 0.9831085205078125),
 ('самый', 0.9828678965568542),
 ('год', 0.9822937846183777),
 ('британский', 0.9821019172668457),
 ('воздушный', 0.9819458723068237),
 ('метрополия', 0.9817474484443665),
 ('штаб', 0.9815644025802612)]

In [24]:
model.wv.most_similar("война")

[('германия', 0.993401288986206),
 ('гитлер', 0.9932308793067932),
 ('англия', 0.9928886294364929),
 ('большой', 0.9921620488166809),
 ('операция', 0.9913949370384216),
 ('время', 0.9913722276687622),
 ('этот', 0.9910632967948914),
 ('ответ', 0.9908947348594666),
 ('лишь', 0.9908230900764465),
 ('союзник', 0.9907991886138916)]

In [25]:
model.wv.most_similar("шторм")

[('цистерна', 0.9990798234939575),
 ('номер', 0.9989914298057556),
 ('пленный', 0.998991072177887),
 ('клапан', 0.9989587068557739),
 ('связывать', 0.9989498257637024),
 ('батарея', 0.9989376664161682),
 ('сбивать', 0.9989270567893982),
 ('ремонт', 0.998914897441864),
 ('возникать', 0.9989051818847656),
 ('политический', 0.9989050626754761)]

In [50]:
model.wv.most_similar("море")

[('труба', 0.994303286075592),
 ('огонь', 0.9941901564598083),
 ('нос', 0.9941226243972778),
 ('подлодка', 0.9937630891799927),
 ('залп', 0.9935479164123535),
 ('черный', 0.9935062527656555),
 ('облако', 0.9934329986572266),
 ('воздушный', 0.9931381940841675),
 ('авиация', 0.9930627942085266),
 ('четыре', 0.9929176568984985)]

In [63]:
model.wv.most_similar("россия")

[('покидать', 0.9978297352790833),
 ('однажды', 0.9977957606315613),
 ('назначение', 0.997729480266571),
 ('мысль', 0.997727632522583),
 ('дух', 0.9976869821548462),
 ('выступать', 0.9976827502250671),
 ('находиться', 0.9976732134819031),
 ('читатель', 0.9976359605789185),
 ('вынуждать', 0.9976322054862976),
 ('вывод', 0.9976314902305603)]